In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
import ee
import io
import tqdm
import json
import fiona
import datetime
import requests
import urllib.request

import numpy as np
import xarray as xr
import pandas as pd
import rsfuncs as rs
import rasterio as rio
import geopandas as gp
import multiprocessing as mp

import matplotlib.pyplot as plt

from tqdm import tqdm
from affine import Affine
from datetime import timedelta
from rasterio import features, mask
from climata.usgs import DailyValueIO
from pandas.tseries.offsets import MonthEnd
from dateutil.relativedelta import relativedelta

ee.Initialize()

In [2]:
# Read catchments, reservoirs
gdf = gp.read_file("../shape/sierra_catchments.shp")
reservoirs_gdf = gp.read_file("../shape/reservoirs_grace.shp")


In [3]:
def get_gcm_dat(shppath, gcm_dir = "../data/CA_BCM/GISS_26", var = "ppt"):

    '''
    Given a path to a shapefile, compute the monthly sums of a variable

    Inputs: 
        shppath (str) - path to shapefile
        gcm_dir - data directory storing the files
        var - name of variable (file suffix)

    Output: (pd.DataFrame) - monthly sum of variable 
    '''

    # Find gcm files
    data_dir = gcm_dir
    files = [os.path.join(data_dir,x) for x in os.listdir(data_dir) if x.endswith(".tiff") if var in x]
    files.sort()

    # Read CVWS shapefile
    with fiona.open(shppath, "r") as shapefile:
        shp_geom = [feature["geometry"] for feature in shapefile]

    # Read the files, mask nans, clip to CVWS, extract dates
    imdict = {}
    outdates = []

    for i in tqdm(files[:]):
        date = datetime.datetime.strptime(i[-12:-5],'%Y_%m')+ timedelta(days=-1) # Get the date 
        datestr = date.strftime('%Y_%m') # Format date
        src = rio.open(i) # Read file
        src2 = rio.mask.mask(src, shp_geom, crop=True) # Clip to shp 
        arr = src2[0] # read as array
        arr = arr.reshape(arr.shape[1], arr.shape[2]).astype(float) # Reshape bc rasterio has a different dim ordering 
        arr[arr < 0 ] = np.nan # Mask nodata vals 
        imdict[datestr] = arr * 0.001 # convert mm to m
        outdates.append(date)
        
    # Stack all dates to 3D array
    var_stacked = np.dstack(list(imdict.values()))

    # Compute monthly sums
    monthsums = []
    for i in range(var_stacked.shape[2]):
        monthsums.append(np.nansum(var_stacked[:,:,i] *275**2 * 1e-9)) # mult by 275^2 m pixel area, convert m^3 to km^3

    outdf = pd.DataFrame(monthsums,outdates)
    outdf.columns = [var]
    return outdf

In [ ]:
for idx, x in enumerate(gdf[8:9].iterrows()):
    print(idx)
    print("****" * 15)
    row  = x[1]
    stn_id = row['stid']
    print(stn_id, row['catch_name'])
    
    # Read the catchment shapefile
    catch_shp = "../shape/catchment_{}.shp".format(stn_id)
    gcm_dir = "../data/CA_BCM/GISS_26"
    
    # Get the GCM data 
    ppt_df = get_gcm_dat(catch_shp, gcm_dir = "../data/CA_BCM/GISS_26", var = "ppt")
    aet_df = get_gcm_dat(catch_shp, gcm_dir = "../data/CA_BCM/GISS_26", var = "aet")
    pet_df = get_gcm_dat(catch_shp, gcm_dir = "../data/CA_BCM/GISS_26", var = "pet")
    
    # Concat the dfs
    mdf = pd.concat([ppt_df, aet_df, pet_df],axis = 1)
    
    # Write
    gcm = os.path.split(gcm_dir)[-1]
    write_dir = os.path.join("../data/GCM_catchdat/", gcm)
    if not os.path.exists(write_dir):
        os.mkdir(write_dir)

    outfn = os.path.join(write_dir,"catch_{}.csv".format(stn_id))
    mdf.to_csv(outfn)

  0%|          | 0/1116 [00:00<?, ?it/s]

0
************************************************************
SHA SACRAMENTO R


100%|██████████| 1116/1116 [06:11<00:00,  3.00it/s]
